In [2]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from tqdm.auto import tqdm

Ponemos valores iniciales

In [ ]:
PCAbool = str("1")
k = str("1")
votPes = str("1")
alpha = str("50")
epsilon = str("1e-2")
outCsv = str("tempOut.csv")

comando = [f'../cpp/tp2']
comando.append(PCAbool)
comando.append(k)
comando.append(votPes)
comando.append(alpha)
comando.append(epsilon)
comando.append("partTrain.csv")
comando.append("partTest.csv")
comando.append(outCsv)

df_part = pd.read_csv("../csv/train.csv", delimiter=',')

proporcion = 0.6

Partimos la base de kaggle train

In [ ]:
df_shuffled = shuffle(df_part)

limite = int(df_shuffled.shape[0] * proporcion)

X_train = df_shuffled.iloc[:limite]
X_test = df_shuffled.iloc[limite:]
X_train.to_csv("../csv/partTrain.csv", index = False)
X_test.to_csv("../csv/partTest.csv", index = False, columns = df_shuffled.columns[1:])

expected = X_test.iloc[:,0]

Primer experimento, variar los epsilon sobre una misma base (posible overfitting)

In [14]:
epsilons = [f"1e-{i}" for i in np.arange(2,20,2)]

with open(f"../csv/expEpsilon.csv", 'a') as f:
    #f.write('conv,epsilon,segundos,accuracy\n')
    for epsilon in tqdm(epsilons):
        comando[5] = str(epsilon)

        result = subprocess.run(comando, stdout=subprocess.PIPE, encoding='ascii')
        conv = int(len(result.stdout.split(' ')) <= 7)

        pred = pd.read_csv("../csv/" + outCsv, delimiter=',')
        pred_labels = np.array(pred['Label'])
        test_labels = np.array(X_test['label'])
        array_preds = np.array([pred_labels, test_labels]).T
        df_preds = pd.DataFrame(array_preds, columns=["pred", "test"])

        tiempo = result.stdout.split(' ')[-2]
        ok = pred_labels == test_labels
        acc = sum(ok) / len(ok)

        print(str(conv) + ' ' + str(epsilon) + ' ' + str(tiempo) + ' ' + str(acc))
        f.write(str(conv) + ',' + str(epsilon) + ',' + str(tiempo) + ',' + str(acc) + '\n')
    f.close()


 11%|█         | 1/9 [00:25<03:22, 25.27s/it]

1 1e-2 24 0.9683333333333334


 22%|██▏       | 2/9 [00:51<03:01, 25.86s/it]

1 1e-4 25 0.9682738095238095


 33%|███▎      | 3/9 [01:17<02:36, 26.02s/it]

...

1 1e-14 27 0.9682738095238095


 89%|████████▉ | 8/9 [03:55<00:33, 33.62s/it]

0 1e-16 45 0.9682738095238095


100%|██████████| 9/9 [05:11<00:00, 34.62s/it]

0 1e-18 75 0.9682738095238095


Para ver si es un tema del x0 inicial aleatorio, vamos a repetir el experimento pero esta vez en un rango de epsilon mas chico (con los epsilon mas altos, los que parecen variar mas sus accuracys) y repetimos 10 veces cada corrida con los mismos parametros (lo unico que va a cambiar son los x0)

In [16]:
cant = 10
epsilons = [f"1e-{i}" for i in np.arange(1,7,1)]

with open(f"../csv/expEpsilon2.csv", 'a') as f:
    #f.write('conv,epsilon,segundos,accuracy\n')
    for epsilon in tqdm(epsilons):
        comando[5] = str(epsilon)
        conv = True
        tiempo = 0
        acc = 0
        for i in np.arange(cant):
            result = subprocess.run(comando, stdout=subprocess.PIPE, encoding='ascii')
            if (len(result.stdout.split(' ')) > 7):
                conv = False

            pred = pd.read_csv("../csv/" + outCsv, delimiter=',')
            pred_labels = np.array(pred['Label'])
            test_labels = np.array(X_test['label'])
            array_preds = np.array([pred_labels, test_labels]).T
            df_preds = pd.DataFrame(array_preds, columns=["pred", "test"])

            tiempo = tiempo + int(result.stdout.split(' ')[-2])
            ok = pred_labels == test_labels
            acc = acc + (sum(ok) / len(ok))

            print(str(int(len(result.stdout.split(' ')) <= 7)) + ' ' + str(epsilon) + ' ' + str(result.stdout.split(' ')[-2]) + ' ' + str(sum(ok)/len(ok)))

        tiempo = str(tiempo / cant)
        acc = str(acc / cant)
        f.write(str(int(conv)) + ',' + str(epsilon) + ',' + tiempo + ',' + acc + '\n')
        print(str(int(conv)) + ',' + str(epsilon) + ',' + tiempo + ',' + acc + '\n')
    f.close()


  0%|          | 0/6 [00:00<?, ?it/s]

1 1e-1 32 0.9680357142857143
1 1e-1 30 0.9680357142857143
1 1e-1 37 0.9680357142857143
1 1e-1 37 0.9680357142857143
1 1e-1 38 0.9680357142857143
1 1e-1 37 0.9680357142857143
1 1e-1 37 0.9680357142857143
1 1e-1 35 0.9680357142857143
1 1e-1 35 0.9680357142857143


 17%|█▋        | 1/6 [06:11<30:58, 371.72s/it]

1 1e-1 37 0.9680357142857143
1,1e-1,35.5,0.9680357142857143

1 1e-2 36 0.9683333333333334
1 1e-2 37 0.9683333333333334
1 1e-2 36 0.9683333333333334
1 1e-2 37 0.9683333333333334
1 1e-2 36 0.9683333333333334
1 1e-2 36 0.9683333333333334
1 1e-2 36 0.9683333333333334
1 1e-2 37 0.9683333333333334
1 1e-2 36 0.9683333333333334


 33%|███▎      | 2/6 [12:32<25:07, 376.92s/it]

...

1 1e-4 36 0.9682738095238095
1,1e-4,37.2,0.9682738095238095

1 1e-5 36 0.9682738095238095
1 1e-5 38 0.9682738095238095
1 1e-5 37 0.9682738095238095
1 1e-5 38 0.9682738095238095
1 1e-5 36 0.9682738095238095
1 1e-5 38 0.9682738095238095
1 1e-5 35 0.9682738095238095
1 1e-5 33 0.9682738095238095
1 1e-5 36 0.9682738095238095


 83%|████████▎ | 5/6 [31:45<06:21, 381.69s/it]

1 1e-5 34 0.9682738095238095
1,1e-5,36.1,0.9682738095238095

1 1e-6 35 0.9682738095238095
1 1e-6 35 0.9682738095238095
1 1e-6 36 0.9682738095238095
1 1e-6 36 0.9682738095238095
1 1e-6 38 0.9682738095238095
1 1e-6 36 0.9682738095238095
1 1e-6 35 0.9682738095238095
1 1e-6 31 0.9682738095238095
1 1e-6 35 0.9682738095238095


100%|██████████| 6/6 [37:53<00:00, 378.89s/it]

1 1e-6 37 0.9682738095238095
1,1e-6,35.4,0.9682738095238095



No parece ser un tema del x0 inicial, posiblemente sea un caso de overfitting. Probemos usando K-fold cross validation:

In [ ]:
from main import kFold_CV

df_train = pd.read_csv("../csv/train.csv", delimiter=',')

epsilons = [f"1e-{i}" for i in np.arange(1,7,1)]
folds = 5

for epsilon in tqdm(epsilons):
    duracion, accuracy, metricas_tot_kfolds, list_df_preds = kFold_CV(df_train,folds,epsilon=str(epsilon))
    print(str(epsilon) + ' ' + str(duracion / folds) + ' ' + str(np.mean(accuracy)))

K-fold cross validation para buscar los mejores parametros para kNN y PCA en un rango acotado por experimentos iniciales

In [ ]:
from main import kFold_CV

df_train = pd.read_csv("../csv/train.csv", delimiter=',')
alphas = np.arange(25,70,2)
ks = np.arange(1,16,2)
folds = 12
for PCAbool in range(2):
    if PCAbool == 1:
        for alpha in alphas:
            for k in ks:
                for votPes in range(2):
                    #kFold_CV(df_train,folds,str(PCAbool),str(k),str(votPes),str(alpha))
                    if alpha == max(alphas) and k == max(ks) and votPes == 0:
                        kFold_CV(df_train,folds,"1",str(k),str(votPes),str(alpha),guardarKMasCercanosBool="1")
                    elif k == max(ks) and votPes == 0:
                        kFold_CV(df_train,folds,"1",str(k),str(votPes),str(alpha),guardarKMasCercanosBool="1",guardarAvectsBool="0",leerAVect="1")
                    else:
                        kFold_CV(df_train,folds,"1",str(k),str(votPes),str(alpha),guardarAvectsBool="0",leerKMasCercanos="1")
    else:
        for k in ks:
            for votPes in range(2):
                if k == max(ks) and votPes == 0:
                    kFold_CV(df_train,folds,"1",str(k),str(votPes),str(alpha),guardarKMasCercanosBool="1",guardarAvectsBool="0",leerAVect="1")
                else:
                    kFold_CV(df_train,folds,"1",str(k),str(votPes),str(alpha),guardarAvectsBool="0",leerKMasCercanos="1")

Hacemos mas zoom en PCA

In [3]:
from main import kFold_CV

df_train = pd.read_csv("../csv/train.csv", delimiter=',')
alphas = np.arange(55,24,-2)
k = 3
folds = 5

for alpha in alphas:
    kFold_CV(df_train,folds,"1",str(k),"0",str(alpha))

100%|██████████| 5/5 [02:13<00:00, 26.78s/it]


Listo!


100%|██████████| 5/5 [02:13<00:00, 26.75s/it]


Listo!


100%|██████████| 5/5 [02:13<00:00, 26.63s/it]


...

Listo!


100%|██████████| 5/5 [01:26<00:00, 17.23s/it]


Listo!


100%|██████████| 5/5 [01:22<00:00, 16.58s/it]

Listo!


Hacemos mas zoom solo kNN

In [5]:
from main import kFold_CV

df_train = pd.read_csv("../csv/train.csv", delimiter=',')
ks = [8,7,6,5,4,3,2,1]
folds = 4

for k in ks:
    for votPes in range(2):
        if k == max(ks) and votPes == 0:
            kFold_CV(df_train,folds,"0",str(k),str(votPes),str(alpha),guardarKMasCercanosBool="1")
        else:
            kFold_CV(df_train,folds,"0",str(k),str(votPes),str(alpha),leerKMasCercanos="1")

100%|██████████| 4/4 [26:49<00:00, 402.42s/it]


Listo!


100%|██████████| 4/4 [00:15<00:00,  3.98s/it]


Listo!


100%|██████████| 4/4 [00:15<00:00,  3.95s/it]


...

Listo!


100%|██████████| 4/4 [00:15<00:00,  3.90s/it]


Listo!


100%|██████████| 4/4 [00:15<00:00,  3.91s/it]

Listo!


PCA vs kNN (tiempo y accuracy en csv, kappa de cohen en consola)

In [11]:
from main import kFold_CV
from main import kappaDeCohen

df_train = pd.read_csv("../csv/train.csv", delimiter=',')
folds = 5

alpha = 35

ls_df_kNN = kFold_CV(df_train,folds,"0","3","0","50")[3] # mejor parametro de kNN

ls_df_PCA = kFold_CV(df_train,folds,"1","3","0",str(alpha))[3] # mejor parametro de PCA

ls_kappa = kappaDeCohen(ls_df_kNN, ls_df_PCA)
print("------------------------------------------------------------------------------")
print(ls_kappa)

         0     1     2      3
0     True  True  True  False
1     True  True  True  False
2     True  True  True  False
3     True  True  True  False
4     True  True  True  False
...    ...   ...   ...    ...
8395  True  True  True  False
8396  True  True  True  False
8397  True  True  True  False
8398  True  True  True  False
8399  True  True  True  False

[8400 rows x 4 columns]
0.9810714285714286
0.9470595238095237
         0     1     2      3
0     True  True  True  False
1     True  True  True  False
2     True  True  True  False
3     True  True  True  False
4     True  True  True  False
...    ...   ...   ...    ...
8395  True  True  True  False
8396  True  True  True  False
8397  True  True  True  False
8398  True  True  True  False
8399  True  True  True  False

[8400 rows x 4 columns]
0.9773809523809524
0.945345096371882
          0     1      2      3
0     False  True  False  False
1     False  True  False  False
2      True  True   True  False
3      True  True   True  F